# Homework 5 - The eternal significance of publications and citations!

This project was carried out by Group 19 of Algorithmic Methods for Data Mining, consisting of:

| NAME and SURNAME | EMAIL |
| --- | --- |
| Pasquale Luca Tommasino | pl.tommasino@gmail.com | 
| Deniz Yilmaz | denizyilmazz@yahoo.com |
| Emmanuele De Lucia | delucia.2099678@studenti.uniroma1.it |
| Paolo Zilviano | zilviano.1916518@studenti.uniroma1.it |


## 1. Data


In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import ijson
import pandas as pd
from tqdm import tqdm
from collections import Counter
import random
import sys

At the first, let's see how the file is structured, let's take the first row of the dataset and see what the columns and the first row of the dataset are:

In [2]:
with open('dblp.v12.json', 'r') as file:
    # Parse the JSON array items one by one
    array_items = ijson.items(file, 'item')
    # Iterate over the JSON array items
    for item in array_items:
        dict1 = item
        break

In [3]:
#List the column of the json file
chiave = list(dict1.keys())
print(chiave)

['id', 'authors', 'title', 'year', 'n_citation', 'page_start', 'page_end', 'doc_type', 'publisher', 'volume', 'issue', 'doi', 'references', 'indexed_abstract', 'fos', 'venue']


In [4]:
#List of values of first row of the json file
valori = list(dict1.values())
print(valori)

[1091, [{'name': 'Makoto Satoh', 'org': 'Shinshu University', 'id': 2312688602}, {'name': 'Ryo Muramatsu', 'org': 'Shinshu University', 'id': 2482909946}, {'name': 'Mizue Kayama', 'org': 'Shinshu University', 'id': 2128134587}, {'name': 'Kazunori Itoh', 'org': 'Shinshu University', 'id': 2101782692}, {'name': 'Masami Hashimoto', 'org': 'Shinshu University', 'id': 2114054191}, {'name': 'Makoto Otani', 'org': 'Shinshu University', 'id': 1989208940}, {'name': 'Michio Shimizu', 'org': 'Nagano Prefectural College', 'id': 2134989941}, {'name': 'Masahiko Sugimoto', 'org': 'Takushoku University, Hokkaido Junior College', 'id': 2307479915}], 'Preliminary Design of a Network Protocol Learning Tool Based on the Comprehension of High School Students: Design by an Empirical Study Using a Simple Mind Map', 2013, 1, '89', '93', 'Conference', 'Springer, Berlin, Heidelberg', '', '', '10.1007/978-3-642-39476-8_19', [2005687710, 2018037215], {'IndexLength': 58, 'InvertedIndex': {'tool.': [42], 'study': [

### Data pre-processing

We now transform the entire *.json file* into a dictionary with selected columns (*id, authors, title, year, n_citation, doc_type, publisher* and *references*)

In [5]:
#Create a dict for Collaboration Graph
dict_entire = {
    'id' : [],
    'authors' : [],
    'title' : [],
    'year' : [],
    'n_citation' : [],
    'doc_type' : [],
    'publisher' : [],
    'references' : []
}

We create functions for processing the *.json file* (they are in **function.py**).

In [6]:
sys.path.append("dict")
from dictionary import *

with open('dblp.v12.json', 'r') as file:
    # Parse the JSON array items one by one
    array_items = ijson.items(file, 'item')
    # Iterate over the JSON array items
    for item in tqdm(array_items):
        # Process each JSON array item as needed
        dict_id(dict_entire, item)
        dict_authors(dict_entire, item)
        dict_title(dict_entire, item)
        dict_year(dict_entire, item)
        dict_n_citation(dict_entire, item)
        dict_doc_type(dict_entire, item)
        dict_publisher(dict_entire, item)
        dict_references(dict_entire, item)

0it [00:00, ?it/s]

4894081it [06:03, 13463.48it/s]


Transform dictionary in Pandas Dataframe:

In [7]:
#Transform dict in dataframe
df_entire = pd.DataFrame(dict_entire)
df_entire.head()

,id,authors,title,year,n_citation,doc_type,publisher,references
0,1091,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",Preliminary Design of a Network Protocol Learn...,2013,1,Conference,"Springer, Berlin, Heidelberg","[2005687710, 2018037215]"
1,1388,[Pranava K. Jha],Further Results on Independence in Direct-Prod...,2000,1,Journal,None,None
2,1674,"[G. Beale, G. Earl]",A methodology for the physically accurate visu...,2011,1,Conference,Eurographics Association,"[1535888970, 1992876689, 1993710814, 203565334..."
3,1688,"[Altaf Hossain, Faisal Zaman, M. Nasser, M. Mu...","Comparison of GARCH, Neural Network and Suppor...",2009,6,Conference,"Springer, Berlin, Heidelberg","[1560724230, 1986968751, 2156909104]"
4,5411,"[Rafael Álvarez, Leandro Tortosa, José-Francis...",COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,2009,0,Conference,None,None


In [8]:
#Save df in csv file
#df_entire.to_csv('df_entire.csv')
#Read csv df file
#df_entire = pd.read_csv('df_entire.csv', index_col=0, low_memory=False)

#Drop NaN value in the column I
#df_entire = df_entire.dropna(subset='id')

#Transform columns from decimal to integer
#df_entire['id'] = df_entire['id'].astype(int)
#df_entire['n_citation'] = df_entire['n_citation'].astype(int)
#Transform column from integer to datetime (year)
#df_entire['year'] = df_entire['year'].astype('datetime64[Y]')

#See the 'head' of dataset
#df_entire.head()


#### 1. Identify the top 10,000 papers with the highest number of citations.

In [9]:
#Create the top 10000 papers with the highest number of citations
df_10th_papers = df_entire.sort_values(by=['n_citation'], ascending=False)[:10000]

#See the first five lines
df_10th_papers.head()

,id,authors,title,year,n_citation,doc_type,publisher,references
4696136,2041404167,[C. E. Shannon],The Mathematical Theory of Communication,1949,48327,Book,None,None
4630907,1639032689,[David E. Goldberg],"Genetic algorithms in search, optimization, an...",1989,44175,Book,None,None
4092588,2912565176,[Lotfi A. Zadeh],Fuzzy sets,1996,42437,None,None,None
2937610,2151103935,[David G. Lowe],Distinctive Image Features from Scale-Invarian...,2004,35541,Journal,Kluwer Academic Publishers,"[19720318, 1541642243, 1560959218, 1676552347,..."
4088311,2911964244,[Leo Breiman],Random Forests,2001,34741,None,None,"[1507255258, 1580948147, 1605688901, 197584664..."


In [10]:
#Save df in csv file
#df_10th_papers.to_csv('df_10th_papers.csv')
#Read csv df file
#df_10th_papers = pd.read_csv('df_10th_papers.csv')

#Transform columns from decimal to integer
#df_10th_papers['id'] = df_10th_papers['id'].astype(int)
#df_10th_papers['n_citation'] = df_10th_papers['n_citation'].astype(int)
#Transform column from integer to datetime (year)
#df_10th_papers['year'] = df_10th_papers['year'].astype('datetime64[Y]')

#### 2. Then the nodes of your graphs would be as follows:

##### - **Citation graph**: you can consider each of the papers as your nodes

In [11]:
#Chech if in the column 'id' there are any NaN value in column
isthere_nan = df_entire['id'].isna().any()
isthere_nan

False

In [12]:
#Create nodes for Citation Graph
citationG_nodes = dict_entire['id']
citationG_nodes[:5]

[1091, 1388, 1674, 1688, 5411]

##### - **Collaboration graph**: the authors of these papers would be your nodes

In [13]:
#Chech if in the column 'authors' there are any NaN value in column
isthere_nan = df_entire['authors'].isna().any()
isthere_nan

True

In [14]:
df_entire = df_entire.dropna(subset='authors')

In [15]:
#Create nodes for Collaboration Graph
collaborationG_nodes_2 = df_entire['authors']

#Unzip the nested lists in one list of authors
collaborationG_nodes = []

for list_1 in collaborationG_nodes_2:
    for author in list_1:
        collaborationG_nodes.append(author)

collaborationG_nodes[:5]

['Makoto Satoh',
 'Ryo Muramatsu',
 'Mizue Kayama',
 'Kazunori Itoh',
 'Masami Hashimoto']

#### 3. For the edges of the two graphs, you would have the following cases:

##### - **Citation graph**: only consider the citation relationship between these 10,000 papers and ignore the rest.

In [16]:
#Take a subset of the 10,000 papers dataset, and transform it in a readable dataset...
##... with only two column, within one value
citationG_edges_df_2 = df_10th_papers.reset_index()[['id','references']]
citationG_edges_df_2 = citationG_edges_df_2.explode('references')
citationG_edges_df_2 = citationG_edges_df_2.dropna(subset=['references']).reset_index()[['id','references']]

#Transform dataset in a list of tuple
citationG_edges = [tuple(x) for x in citationG_edges_df_2.to_records(index=False)]
citationG_edges[:5]

[(2151103935, 19720318),
 (2151103935, 1541642243),
 (2151103935, 1560959218),
 (2151103935, 1676552347),
 (2151103935, 1681491849)]

##### - **Collaboration graph**: only consider the collaborations between the authors of these 10,000 papers and ignore the rest.

In [17]:
collaboration_dict = {
    'name1' : [],
    'name2' : []
}

In [18]:
#Take a subset of the 10,000 papers dataset, and transform it in a readable dataset...
##... with only two column, within one value
collaborationG_edges_df_2 = df_10th_papers.reset_index()[['authors']]
nestedlist_auth_collaboration = collaborationG_edges_df_2['authors'].tolist()

#Create all possible matches of collaboration for each book
collaborationG_edges = []
for authors_list in nestedlist_auth_collaboration:
    for i in range(len(authors_list)):
        for j in range(i + 1, len(authors_list)):
            collaborationG_edges.append((authors_list[i], authors_list[j]))

collaborationG_edges[:5]


[('David W. Hosmer', 'Stanley Lemeshow'),
 ('Chih-Chung Chang', 'Chih-Jen Lin'),
 ('Corinna Cortes', 'Vladimir Vapnik'),
 ('Heng Li', 'Richard Durbin'),
 ('K. Deb', 'A. Pratap')]

In [19]:
#Create a counter for couple
collaborationG_edges_counter = Counter(collaborationG_edges)

#Create a dictionary for track couple weight
collaborationG_edges_weights = {}

#Iteract edges and count from dict
for edge, count in collaborationG_edges_counter.items():
    collaborationG_edges_weights[edge] = count

    #Add weight also for inverted edge
    inverted_edge = (edge[1], edge[0])
    collaborationG_edges_weights[inverted_edge] = count

#### Now we build the graphs:

##### - **Citation graph**:

In [20]:
#Create a unweighted and directed graph
citation_graph = nx.DiGraph()

# Add nodes to the graph
citation_graph.add_nodes_from(citationG_nodes)
# Add edges to the graph
citation_graph.add_edges_from(citationG_edges)

In [21]:
#Save in txt
#file_path = "citation_graph.txt"
#nx.write_edgelist(citation_graph, file_path)

##### - **Collaboration graph**:

In [22]:
#Create a weighted and undirected graph
collaboration_graph = nx.Graph()

#Add nodes to the graph
collaboration_graph.add_nodes_from(collaborationG_nodes)

#Add edges to the graph
for edge, weight in collaborationG_edges_weights.items():
    collaboration_graph.add_edge(edge[0], edge[1], weight=weight)

In [23]:
#Save in txt
#file_path = "collaboration_graph.txt"
#nx.write_edgelist(collaboration_graph, file_path)

# 2. Controlling system


### 2.1 Backend Implementation


In [24]:
#For this section (2.1) we work with the subgraph (10,000) of the entire graph

#Shortlist for Citation Subgraph
books_red = df_10th_papers['id'].tolist()
#Subgraph
sub_citation_graph = citation_graph.subgraph(books_red)

#Create the shortlist for Collaboration Subgraph
auth_red = []
for list_2 in nestedlist_auth_collaboration:
    for auth in list_2:
        auth_red.append(auth)
#Subgraph
sub_collaboration_graph = collaboration_graph.subgraph(auth_red)

#### Functionality 1 - Graph's features

**Input**:
- The graph
- The name of the graph

**Output**:
- The number of the nodes in the graph
- The number of the edges in the graph
- The graph density
- The graph degree distribution
- The average degree of the graph
- The graph hubs (hubs are nodes having degrees more extensive than the 95th percentile of the degree distribution)
- Whether the graph is dense or sparse

In [25]:
sys.path.append("funct")
from functionality1 import functionality_1

name_graph = 'Sub Citation Graph'
len_nodes_citation, len_edges_citation, density_citation, degree_sequence_citation, avg_degree_citation, hubs_citation, classification_graph_citation = functionality_1(sub_citation_graph, name_graph)

In [26]:
sys.path.append("funct")
from functionality1 import functionality_1

name_graph = 'Sub Collaboration Graph'
len_nodes_collab, len_edges_collab, density_collab, degree_sequence_collab, avg_degree_collab, hubs_collab, classification_graph_collab = functionality_1(sub_collaboration_graph, name_graph)

#### Functionality 2 - Nodes' contribution

##### Input:
- The graph
- A node of the graph (paper/author)
- The name of the graph

##### Output:

- The centrality of the node, calculated based on the following centrality measurements:
    - Betweeness
    - PageRank
    - ClosenessCentrality
    - DegreeCentrality

In [27]:
sys.path.append("funct")
from functionality2 import functionality_2

#node_selected_collaboration = 2155511848

name_graph = 'Sub Citation Graph'
#node_selected = int(input("Insert the Node that the function calculated: "))
betw, pagerank, closeness, degree = functionality_2(sub_citation_graph, node_selected=2155511848, name_graph=name_graph)

#### Functionality 3 - Shortest ordered walk

##### Input:
- The graph data
- A sequence of authors_a = [a_2, ..., a_{n-1}]
- Initial node a_1 and an end node a_n
- *N*: denoting the top *N* authors whose data should be considered

##### Output:
- The shortest walk of collaborations you need to read to get from author a_1 to author a_n and the papers you need to cross to realize this walk.


Considerations:
For this functionality, you must implement an algorithm that returns the shortest __walk__ that goes from node a\_j to a\_n, which visits **in order** the nodes in _a_. The choice of a\_j and a\_n can be made randomly (or if it improves the performance of the algorithm, you can also define it in any other way)

__Important Notes:__
- This algorithm should be run only on the collaboration graph.
- The algorithm needs to handle the case that the graph is not connected. Thus, only some nodes in _a_ are reachable from a\_1. In such a scenario, it is enough to let the program give in the output the string "There is no such path."
- Since we are dealing with walks, you can pass on the same node _a\_i_ more than once, but you must preserve order. It means you can go back to any author node any time you want, assuming that the order in which you visit the required nodes is still the same.
- Once you completed your implementation, ask chatGPT for a different one leveraging another approach in solving the shortest path and prove whether this implementation is correct.

In [28]:
sys.path.append("funct")
from functionality3 import functionality_3

#We selected N = 100
N_sel = 100

total_shortest_walk, path = functionality_3(sub_collaboration_graph, sequence_authors=0, node_first=0, node_last=0, N=N_sel)

print(total_shortest_walk)
print(path)

The first node is: Benoit Marchand
The last node is: Eri Saijyo
The authors sequence is: ['Hideki Tatsukawa', 'Timo Lassmann', 'Andrew P. Gibson', 'Boris R. Jankovic', 'Naoko Suzuki', 'Christian Schmidl', 'Mariko Okada-Hatakeyama', 'Ilya E. Vorontsov']
9
['Benoit Marchand', 'Hideki Tatsukawa', 'Timo Lassmann', 'Andrew P. Gibson', 'Boris R. Jankovic', 'Naoko Suzuki', 'Christian Schmidl', 'Mariko Okada-Hatakeyama', 'Ilya E. Vorontsov', 'Eri Saijyo']


- Once you completed your implementation, ask chatGPT for a different one leveraging another approach in solving the shortest path and prove whether this implementation is correct. 

In [28]:
import networkx as nx

def shortest_ordered_walk(graph, authors_a, initial_node, end_node, N):
    # Filter the graph to consider only the top N authors
    filtered_graph = filter_graph_by_top_N(graph, N)
    
    # Initialize variables
    shortest_walk = None
    shortest_path_length = float('inf')
    
    # Iterate through all possible pairs (a_j, a_n)
    for i in range(len(authors_a) - 1):
        current_node = authors_a[i]
        next_node = authors_a[i + 1]
        
        # Find the shortest path from current_node to next_node
        path = nx.shortest_path(filtered_graph, source=current_node, target=next_node)
        path_length = len(path)
        
        # Update the shortest walk if a shorter path is found
        if path_length < shortest_path_length:
            shortest_walk = path
            shortest_path_length = path_length
    
    return shortest_walk

def filter_graph_by_top_N(graph, N):
    # For demonstration purposes, let's consider the top N nodes based on degree centrality
    top_nodes = sorted(graph.nodes(), key=lambda x: graph.degree(x), reverse=True)[:N]
    
    filtered_graph = graph.subgraph(top_nodes)
    
    return filtered_graph

result = shortest_ordered_walk(sub_collaboration_graph, authors_a=['Leonard Lipovich', 'Yishai Shimoni', 'Marco Roncador', 'Naoko Takahashi Tagami', 'Giuseppe Jurman', 'Albin Sandelin', 'Masanori Suzuki', 'Linda M. Van Den Berg'], initial_node='Boris Lenhard', end_node='Anita Schwegmann', N=100)
print("Shortest Ordered Walk:", result)

NodeNotFound: Either source Leonard Lipovich or target Yishai Shimoni is not in G

Premise: in ChatGPT chat, we were required to implement the code following the Functionality 3 guidelines described as above.

The answer is obviously not suitable in our case since ChatGPT does not propose in any way to follow the *start node* and *end nodes* but tries to take the *shortest path* in the proposed *sequence*. In addition, as in the example above, it fails to handle whether a node is missing

#### Functionality 4 - Disconnecting Graphs

##### Input:

- The graph data
- authorA: a paper to which will relate sub-graph G_a
- authorB: a paper to which will relate sub-graph G_b
- *N*: denoting the top *N* authors that their data should be considered

##### Output:

- The minimum number of edges (by considering their weights) required to disconnect the original graph in two disconnected subgraphs: G_a and G_b.

In [29]:
sys.path.append("funct")
from functionality4 import functionality_4

#We selected N = 100
N_sel = 100

#If we put 0 in 'start' and 'target' we selected two random authors
removed_edges = functionality_4(sub_collaboration_graph, start=0, target=0, N=N_sel)

#### Functionality 5 - Extracting Communities

##### Input:

- The graph data
- *N*: denoting the top *N* papers that their data should be considered
- Paper_1: denoting the name of one of the papers
- Paper_2: denoting the name of one of the papers

##### Output:

- The minimum number of edges that should be removed to form communities
- A list of communities, each containing a list of papers that belong to them.
- Whether the Paper_1 and Paper_2 belongs to the same community.

In [4]:
#sys.path.append("funct")
#from functionality5 import functionality_5

'''
#number of nodes we want to analyze
n=2

#subsample = random.sample(list(sub_citation_graph.nodes), n)
print("Papers we want:", subsample[0], subsample[1])


def functionality_5(graph, p1, p2):
    #We need an undirected graph
    graph_copy = graph.to_undirected()

    if nx.number_connected_components(graph_copy) > 1:
        print("The graph is not full connected!")
        return 0, [], False
    
    #Calculate the initial number of edges
    num_edges_start = graph_copy.number_of_edges()

    #Calculate betweenness centrality for each edge
    edge_betweenness = nx.edge_betweenness_centrality(graph_copy)

    # Sort edges by betweenness centrality in descending order
    sorted_edges = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)

    #Remove edges until the graph has more than one connected component 
    while nx.number_connected_components(graph_copy) ==1:
        edge_to_remove = sorted_edges.pop(0)[0]
        graph_copy.remove_edge(*edge_to_remove)
    
    #Find connected components as communities
    communities = list(nx.connected_components(graph_copy))

    # Check if Paper_1 and Paper_2 belong to the same community
    same_community = any({p1,p2} <= community for community in communities)

    #Calculate the minimum number of edges removed
    num_edges_end = graph_copy.number_of_edges()

    diff = num_edges_start-num_edges_end

    # Calculate the minimum number of edges removed
    return diff, communities, same_community

min, cm, is_cm = functionality_5(sub_citation_graph.copy(), 2152761983, 2088032561)#subsample[0], subsample[1])
print("Edges to eliminate:", min) 
print("Number of communities:", len(cm))
print("Are the paper in the same community?", is_cm)
'''

'\n#number of nodes we want to analyze\nn=2\n\n#subsample = random.sample(list(sub_citation_graph.nodes), n)\nprint("Papers we want:", subsample[0], subsample[1])\n\n\ndef functionality_5(graph, p1, p2):\n    #We need an undirected graph\n    graph_copy = graph.to_undirected()\n\n    if nx.number_connected_components(graph_copy) > 1:\n        print("The graph is not full connected!")\n        return 0, [], False\n    \n    #Calculate the initial number of edges\n    num_edges_start = graph_copy.number_of_edges()\n\n    #Calculate betweenness centrality for each edge\n    edge_betweenness = nx.edge_betweenness_centrality(graph_copy)\n\n    # Sort edges by betweenness centrality in descending order\n    sorted_edges = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)\n\n    #Remove edges until the graph has more than one connected component \n    while nx.number_connected_components(graph_copy) ==1:\n        edge_to_remove = sorted_edges.pop(0)[0]\n        graph_copy.rem

### 2.2. Frontend Implementation

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def display_menu():
    print("==== Menu ====")
    print("1. Display Graph Features")
    print("2. Display Node Contribution")
    print("3. Display Shortest Ordered Path")
    print("4. Display Disconnected Graph")
    print("5. Display Communities")
    print("6. Perform Operation A")
    print("7. Perform Operation B")
    print("8. Display Statistics")
    print("9. Exit")

def get_user_choice():
    try:
        choice = int(input("Enter your choice (1-9): "))
        if 1 <= choice <= 9:
            return choice
        else:
            print("Invalid choice. Enter a number between 1 and 9.")
            return get_user_choice()
    except ValueError:
        print("Invalid input. Enter a valid number.")
        return get_user_choice()

def display_results(results):
    print("\n==== Results ====")
    print(results)
    print("=================")

def get_graph_data(graph_name):
    if graph_name == "Graph1":
        G = nx.Graph()
        # Add nodes and edges to G - replace with your actual data or logic
        return G
    elif graph_name == "Graph2":
        G = nx.Graph()
        # Add nodes and edges to G - replace with your actual data or logic
        return G
    else:
        raise ValueError("Invalid graph name")

def visualization_1(graph_name):
    G = get_graph_data(graph_name)

    print("==== Visualization 1: Visualize Graph Features ====")

    # Call backend functionality_1
    functionality_1(G, graph_name)

    # Additional visualizations for citations and collaborations
    # Plot distribution of citations received
    plt.figure(figsize=(10, 6))
    degrees_citation = dict(G.degree())
    plt.hist(list(degrees_citation.values()), bins=20, color='blue', alpha=0.7)
    plt.title("Distribution of Citations Received by Papers")
    plt.xlabel("Degree (Number of Citations)")
    plt.ylabel("Number of Papers")
    plt.show()

    # Plot distribution of given citations
    plt.figure(figsize=(10, 6))
    in_degrees_citation = dict(G.in_degree())
    plt.hist(list(in_degrees_citation.values()), bins=20, color='green', alpha=0.7)
    plt.title("Distribution of Citations Given by Papers")
    plt.xlabel("In-Degree (Number of Citations Given)")
    plt.ylabel("Number of Papers")
    plt.show()

    # Plot number of collaborations of the author
    plt.figure(figsize=(10, 6))
    degrees_collaboration = dict(G.degree())
    plt.hist(list(degrees_collaboration.values()), bins=20, color='orange', alpha=0.7)
    plt.title("Number of Collaborations of the Author")
    plt.xlabel("Degree (Number of Collaborations)")
    plt.ylabel("Number of Authors")
    plt.show()

def visualization_2(graph_name):
    G = get_graph_data(graph_name)

    print("==== Visualization 2: Visualize the Node's Contribution ====")

    # Select a node from the graph
    node_selected = input("Enter the node to analyze: ")

    # Call backend functionality_2
    functionality_2(G, node_selected, graph_name)


# Main program loop
while True:
    display_menu()
    choice = get_user_choice()

    if choice == 1:
        graph_name = input("Enter the name of the graph (Graph1 or Graph2): ")
        visualization_1(graph_name)
    elif choice == 2:
        # Implement other functionalities as needed
        # ...
    elif choice == 9:
        print("Exiting the program. Goodbye!")
        break
    else:
        print("Invalid choice. Please select a valid option.")



# 4. Command Line Question (CLQ)


It is possible to find the exercise in the appropriate `CommandLine.sh` file, and the screenshot in the `CLQ_screenshots` folder 

# 5. Algorithmic Question (AQ)

---
## Part A
### 1. Implement an algorithm to solve the described mentioned problem.

In [ ]:
nms = list(map(int, input().split()))
n = nms[0]
m = nms[1]
s = nms[2]

sports = list(map(str, input().split()))
assert len(sports) == m, f"We need a set of {m} sports."
assert all(len(s) == 3 and s.isalpha() and s.isupper() for s in sports), "Every element of the list must be a 3-character string with only uppercase letters."

athl = dict()

for i in range(n):
    ix = int(input()) #ix = int(input('id of the athlete: '))
    athl[ix] = {} #initializing a dictionary for each athlete which
                  #will contain as key the sport and as value the proficiency of that skill
    for j in range(s):
        l = list(map(str, input().split()))
        athl[ix][str(l[0])] = int(l[1])

ovr = 0  #initializing the overall score
tkn = list() #list of athletes (ids) who are already participating for a determined sport
for i in sports:
    mx = 0
    for j in range(1, n + 1):
        if i in athl[j].keys() and athl[j][i] > mx and j not in tkn:
            mx = athl[j][i]
            tk = j
    ovr += mx
    tkn.append(tk)
    if len(tkn) == m:
        break

print(ovr)

# Input 1 expected output:  370

14 10 1
SWM VOL ATH VOL VOL BSK HCK BSK SWM BSK
1
BSK 98
2
ATH 14
3
HCK 82
4
HCK 9
5
FTB 90
6
ATH 52
7
HCK 95
8
TEN 85
9
RGB 46
10
SWM 16
11
VOL 32
12
SOC 41
13
SWM 59
14
SWM 34
370


In [ ]:
nms = list(map(int, input().split()))
n = nms[0]
m = nms[1]
s = nms[2]

sports = list(map(str, input().split()))
assert len(sports) == m, f"We need a set of {m} sports."
assert all(len(s) == 3 and s.isalpha() and s.isupper() for s in sports), "Every element of the list must be a 3-character string with only uppercase letters."

athl = dict()

for i in range(n):
    ix = int(input()) #ix = int(input('id of the athlete: '))
    athl[ix] = {} #initializing a dictionary for each athlete which
                  #will contain as key the sport and as value the proficiency of that skill
    for j in range(s):
        l = list(map(str, input().split()))
        athl[ix][str(l[0])] = int(l[1])

ovr = 0  #initializing the overall score
tkn = list() #list of athletes (ids) who are already participating for a determined sport
for i in sports:
    mx = 0
    for j in range(1, n + 1):
        if i in athl[j].keys() and athl[j][i] > mx and j not in tkn:
            mx = athl[j][i]
            tk = j
    ovr += mx
    tkn.append(tk)
    if len(tkn) == m:
        break

print(ovr)

# Input 2 expected output:  399

14 10 2
SWM VOL ATH VOL VOL BSK HCK BSK SWM BSK
1
BSK 98
HCK 12
2
ATH 14
VOL 1
3
HCK 82
ATH 30
4
HCK 9
SWM 27
5
FTB 90
HCK 50
6
ATH 52
RGB 80
7
HCK 95
SWM 11
8
TEN 85
RGB 7
9
RGB 46
SWM 30
10
SWM 16
BSK 12
11
VOL 32
HCK 40
12
SOC 41
FTB 12
13
SWM 59
TEN 82
14
SWM 34
VOL 20
399


---
### 2. What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.

In the first for loop we are iterating on $n$ to initialize the dictionary for each athlete (`len(athl) = n`) and inside this loop there is another one which iterates on $s$ to take as imput all the proficiencies in each of the $s$ sport for each athlete.
Then in the final big for loop we are iterating on the list "sport" which has `len(sports) = m` , while inside we iterate again on the $n$ athletes. So if we calculate the big O time complexity it will be:

\begin{align}
O(n \cdot s) + O (m \cdot n) = O(n \cdot s + m \cdot n) = O(n \cdot (m+s))
\end{align}

---
### 3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. If the two differ, which one is right? (why?)

We asked ChatGPT and this was the answer:


Let's analyze the time complexity of the provided algorithm.

1. **Input Reading:**
   - Reading the number of athletes (`n`), athletes to be chosen (`m`), and the number of sports skills per athlete (`s`) takes constant time, O(1).

2. **Sports Input:**
   - Reading the list of sports skills (`sports`) takes O(s) time, where s is the number of sports skills.

3. **Athlete Information Input:**
   - Reading information about each athlete (athlete ID, sports skills, and proficiency) involves a nested loop. The outer loop runs 'n' times (number of athletes), and the inner loop runs 's' times (number of sports skills per athlete). So, the time complexity of this part is O(n ⋅ s).

4. **Sport Selection Loop:**
   - The outer loop runs 's' times (number of sports). In each iteration, there is an inner loop over all athletes ('n' athletes) to find the athlete with the highest proficiency in the current sport. In the worst case, each athlete is considered for each sport, resulting in a time complexity of O(s ⋅ n).

5. **Overall Complexity:**
   - Combining the complexities of the above steps, the overall time complexity is O(1) + O(s) + O(n ⋅ s) + O(s ⋅ n), which simplifies to O(n ⋅ s).

Therefore, the big O complexity of this algorithm is O(n ⋅ s), where 'n' is the number of athletes, and 's' is the number of sports skills per athlete.


### Giving ChatGPT the code it assumed that the length of the list of the m sports was s, so this answer is wrong!

---
### 4. If you algorithm has exponential time complexity, can you provide a polynomial-time version?
The algorithm has not exponential complexity

### 5. If S = 1, how does the time complexity of an optimal algorithm to solve this problem change?
If we watch the answer above in point 1 we can see that if we replace s with 1 the equation would be:

\begin{align}
O(n \cdot 1) + O (m \cdot n) = O(n + m \cdot n) = O(m \cdot n)
\end{align}

---
## Part B

### 1. Prove or disprove that the problem is NP-complete.


**Problem Overview**:

This problem involves two main tasks:

Ensuring that a team covers all the required skills in the set $T$.
Minimizing the total effort required for the team members to work well together.
Proving NP-Completeness:
To show that the problem is NP-complete, we need to demonstrate two things:

1. The problem is in NP (verifiable in polynomial time):
For each skill in the set $T$, we need to check if there is at least one team member in $V$' who possesses that skill. This verification can be done by going through the set T and the set V', and the time it takes is proportional to the product of the sizes of $T$ and $V'$, denoted as $O(T  ⋅ V')$.

Secondly, we use a minimum spanning tree to check if $V'$ is minimized. This involves verifying if the tree connects all the vertices in $V'$ without forming any cycles. Additionally, we check if the total weight of the tree is less than or equal to any other possible form of $G[V']$. Kruskal's algorithm can be employed for this, and its time complexity is $O(V' log V)$.

Both of these verification steps take polynomial time, confirming that the problem is in NP.

2. The problem is NP-Hard:
This is typically demonstrated by reducing a known NP-complete problem to the given problem, but this part is not explicitly mentioned in the provided text.
In summary, the problem is in NP because we can verify a solution in polynomial time, but the proof of NP-hardness is not explicitly discussed in the provided text.








### 2. Write a heuristic in order to approximate the best solution for this problem.


1. Set up a Team Subset (V'):
    Start with an empty group of individuals, which we'll call $V'$.
    

2. Skill-Based Team Building:
    For each skill in the required set of skills ($T$), pick an individual (vertex $v$) who has that skill. Choose the one that has the least cumulative effort in working with individuals already in the team ($V'$). If there are multiple individuals with the same skill, prioritize the one that adds the least additional effort.
    
    

3. Build the Collaboration Graph ($G$):
    Create an undirected weighted graph ($G$) where the nodes represent individuals in the team subset ($V'$) and the edges represent pairs of individuals. The weights on the edges represent the effort required to work together based on their skills.
     
     

4. Complete the Team (V'):
    Continue the process until all the required skills in $T$ are covered by individuals in the team subset ($V'$).
    


5. Apply Kruskal's Algorithm:
    Use the Kruskal algorithm, as mentioned before, to identify the subset of edges in the graph ($E'$) that connect individuals in the team subset ($V'$) with the least overall weight.
    
    

4. Resulting Collaboration Team:
    The final team subset ($V'$) and the selected edges ($E'$) form a connected group of individuals who collectively cover all the required skills with minimal effort in working together.
    
    
    
This approach ensures that the team is composed of individuals who not only possess the necessary skills but also collaborate efficiently based on their skill compatibility.








### 3. What is the time complexity of your solution?

Since the first step has time-complexity $O(T ⋅ V)$ and the second step has $O(V' log V)$ and in this type of problem $T$ is most of the times larger than $log V$, the complexity will be $O(T ⋅ V)$.